In [2]:
import sys
print( sys.version )
import open3d as o3d

3.9.13 (main, May 23 2022, 22:01:06) 
[GCC 9.4.0]


ModuleNotFoundError: No module named 'open3d'

In [ ]:
pcd = o3d.io.read_point_cloud("data/bunny.pcd")

In [ ]:
print(pcd)

In [ ]:
import numpy as np
import open3d as o3d
from open3d.visualization import Visualizer
# from open3d.visualization import JVisualizer
# from open3d import JVisualizer


points = (np.random.rand(1000, 3) - 0.5) / 4
colors = np.random.rand(1000, 3)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([pcd])


In [ ]:
import numpy as np
import open3d as o3d

o3d.visualization.RenderOption.line_width=16.0

N = 100
n = N-1

vertex = o3d.utility.Vector3dVector( (np.random.rand(N, 3) - 0.5)/4 )
endPnt = []
for i in range(n):
    endPnt.append( [i, i+1] )
endPnt  = o3d.utility.Vector2iVector( endPnt )
segment = o3d.geometry.LineSet( vertex, endPnt )

o3d.visualization.draw_geometries( [ segment ] )


In [ ]:
# http://www.open3d.org/docs/0.6.0/python_api/open3d.geometry.Geometry3D.html?highlight=transform#open3d.geometry.Geometry3D.transform
cyl = o3d.geometry.TriangleMesh.create_cylinder(radius=1.0, height=2.0, resolution=20, split=4)
# dir(cyl)

In [ ]:
o3d.visualization.draw_geometries([cyl])

In [ ]:
# examples/python/visualization/non_blocking_visualization.py

import open3d as o3d
import numpy as np
import copy

if __name__ == "__main__":
    # 1. Load data and set model poses
    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
    source_raw = o3d.io.read_point_cloud( "data/cloud_bin_0.pcd" )
    target_raw = o3d.io.read_point_cloud( "data/cloud_bin_1.pcd" )
    source = source_raw.voxel_down_sample(voxel_size=0.02)
    target = target_raw.voxel_down_sample(voxel_size=0.02)
    trans = [ [ 0.862, 0.011, -0.507,  0.0] , 
              [-0.139, 0.967, -0.215,  0.7] ,
              [ 0.487, 0.255,  0.835, -1.4] , 
              [ 0.0  , 0.0  ,  0.0  ,  1.0] ]
    source.transform(trans)

    flip_transform = [ [ 1,  0,  0, 0 ] , 
                       [ 0, -1,  0, 0 ] , 
                       [ 0,  0, -1, 0 ] , 
                       [ 0,  0,  0, 1 ] ]
    
    source.transform( flip_transform )
    target.transform( flip_transform )

    # 2. Init vis window
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(source)
    vis.add_geometry(target)
    threshold = 0.05
    icp_iteration = 100
    save_image = False

    # 3. Animation loop
    for i in range( icp_iteration ):
        reg_p2l = o3d.pipelines.registration.registration_icp(
            source, target, threshold, np.identity(4),
            o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1))
        source.transform(reg_p2l.transformation)
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        if save_image:
            vis.capture_screen_image( "temp_%04d.jpg" % i )
            
    # 4. Loop over, destroy window
    vis.destroy_window()